# BiSRe-LLM: Perturbation Experiment with Meta-Llama-3-8B-Instruct

In this notebook I set up the perturbation experiments for meta-llama/Meta-Llama-3-8B-Instruct. First, the general infrastructure for interacting with Llama3 is set up. Second, the prompt is designed as well as the iterative API requests to Huggingface to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Dataset

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
# define the relevant columns
test_IDs = ["Q71", "Q182", "Q210", "Q97", "Q228", "Q37", "Q242", "Q142", "Q6", "Q63"]

main_question = ["question"]
main_answer = ["list_answer_options"]

bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle","list_answer_options_reversed", "list_answer_options"]
non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased"]

# read the preprocessed questionnaire
questionnaire =  pd.read_csv('../assets/20240814_final_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["full_question_ID"].isin(test_IDs)]
print(questionnaire.shape)
questionnaire.head()

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

## Huggingface API Inference

In [27]:
from huggingface_hub import InferenceClient

import requests

API_URL = "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct"

file = open("..\..\Administration\HF_token.txt", "r")
token = file.read()
headers = {"Authorization": "Bearer "+ token}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [40]:
# create empty response list which is transformed to a dataframe
response_list = []

# define rounds to be tested
rounds = 20

for round in range(0,rounds):
  print("Current round: ", round)
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          # Return: list[Recipe]"""
          prompt = f"""
            Answer the following question: {question}
            This is a list of possible answer options: {answer_options}
            You must pick one of the answer options. Only answer with the label.
            """
          #print("Prompt: ", prompt)
          raw_response = query({"inputs": prompt,
                "parameters": {
                  "return_full_text": False,
                  "max_new_tokens": 4,
                  "temperature": 1.0}
                })
          response = raw_response[0]["generated_text"]
          #print(response)
          
          df_response_input = [round, question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              
              # Return: list[Recipe]"""
              prompt = f"""
                Answer the following question: {question}
                This is a list of possible answer options: {answer_options}
                You must pick one of the options. Only answer with the label.
                """
              #print("Prompt: ", prompt)
              raw_response = query({"inputs": prompt,
                "parameters": {
                  "return_full_text": False,
                  "max_new_tokens": 4,
                  "temperature": 1.0}
                })
              response = raw_response[0]["generated_text"]
              #print(response)
              df_response_input = [round, question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)

Current round:  0
Current round:  1
Current round:  2
Current round:  3


KeyError: 0

In [41]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Meta-Llama-3-8B-Instruct"

In [42]:
model_response.head()

,round,question_ID,question,answer_options,type,response,model_name
0,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",list_answer_no_refusal,\n\n\n\n```\n1,Meta-Llama-3-8B-Instruct
1,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",list_answer_no_middle,\n\n\n\n```\n1,Meta-Llama-3-8B-Instruct
2,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",list_answer_options_reversed,Very important \n,Meta-Llama-3-8B-Instruct
3,0,Q6,"Important in life: Religion For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]",list_answer_options,\n\n\n\n```\n1,Meta-Llama-3-8B-Instruct
4,0,Q6,"Important in lofe: Religion For each of the following aspects, indicate how important il is in your life. Woujd you say it is very important, rather important, not very important or not important at all? Religion","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]",question_typo,\n\n\n# Answer:,Meta-Llama-3-8B-Instruct


In [43]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/test_interviews/{current_date}_llama3_test_questions.csv", encoding="utf-8", sep=";")

## Experiments on Remote PC/BWUniCluster

The following section contains the experimental setup and scripts conducted on the remote PC.

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

#bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle", "list_answer_middle", "list_answer_options_reversed", "list_answer_options", "original_personality_prime", "original_emotional_prime"]
#non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym","question_paraphrased", "question_keyboardtypo"]

# delayed experiments with middle category
bias_answer_columns = ["list_answer_middle"]
non_bias_question_columns = []

# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20241029_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire.head(2)

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
0,0,Q1,"Important in life: Family For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Family",Q1,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ic life: Fimily nor eaca jf the following aspecto, indicete how important it ir wn your liae. Would bou say it is very imporcant, rathes importano, not very imporgant ob not impodtant at all? Famizy","Impartont in life: Flmiay For each of the following sapects, indicate how important it is in your life. uoWld you yas it is evry important, ratreh important, ton very important or not important ta all? Family","Crucial in life: Family For each of the following aspects, indicate how significant it is in your life. Would you say it is very important, rather important, not very important or not at all important? Family","How important is family to you? Please rate its significance in your life on a scale of ""very important"" to ""not important at all"".","Imporyant in kife: Family For esch of rhe following aspects, indicate how important ut is un your lide. Would you say it ia very imporrant, rather importamt, nit very important ir nor important ay all? Fanily",1
1,1,Q2,"Important in life: Friends For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? Friends",Q2,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all']",4,False,"['1=Very important ', '2=Rather important ', '3=Not very important ', '4=Not important at all', ""-1=Don't know""]","['1=Very important ', '2=Rather Important ', '3=Neutral', '4=Not very important ', '5=Not important at all', '-1=Don't know']","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[1=Very important , 2=Rather important , 3=Not very important , 4=Not important at all, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '4=Not important at all', '3=Not very important ', '2=Rather important ', '1=Very important ']",both,"Important ix life: Fpiends For xach of the following aspects, indicate how important iv iu in kour lifev Would dou say it os very importcnt, rather impoutant, nom vwry important hr not important xt bll? Friemds","Important ni :ifel Frisnde For each of the following aspects, indicaet 

In [4]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=0,
)

messages = [
    #{"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

answer = pipe(
    messages,
    max_new_tokens=10,
)
print(answer[0]["generated_text"][-1])

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{'role': 'assistant', 'content': "I'm an artificial intelligence model known as Llama"}


In [5]:
# create empty response list which is transformed to a dataframe

response_list = []

# define rounds to be tested
rounds = 25

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          messages = [
              {"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
          answer = pipe(messages, max_length=1000, max_new_tokens=10)
          response = answer[0]["generated_text"][-1]["content"]
          print(response)
          
          df_response_input = [round,question_ID, question, answer_options, k,  response]
          response_list.append(df_response_input)
          torch.cuda.empty_cache()
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              messages = [
                  {"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]
              answer = pipe(messages, max_length=1000, max_new_tokens=10)
              response = answer[0]["generated_text"][-1]["content"]
              print(response)
              df_response_input = [round,question_ID, question, answer_options, n, response]
              response_list.append(df_response_input)
              torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1=Very important


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

11


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


6=Somewhat essential


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


11=Essential feature
-1=Don't know


In [6]:
print("Length of responses: ", len(response_list))

Length of responses:  1550


In [7]:
# create a bias interaction experiment where there is a question-side and answer option-side bias
# here we use
perturbed_question = "list_answer_options_reversed"
perturbed_answer = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  torch.cuda.empty_cache()
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      # first retrieve original question
      question = questionnaire.iloc[row]["question_paraphrased"]
      # second retrieve bias perturbed answer options from the questionnaire  
      answer_options = questionnaire.iloc[row]["list_answer_options_reversed"]      
      messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]    
      answer = pipe(messages, max_length=1000, max_new_tokens=10)       
      response = answer[0]["generated_text"][1]["content"]         
      print(response)  
      df_response_input = [round,question_ID, question, answer_options, "interaction_paraphrase_reversed",  response]
      response_list.append(df_response_input)
      torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Current round:  0


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

5


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


5


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=10) and `max_length`(=1000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


9
1=Not an essential feature


In [8]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  3100


In [9]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "Meta-Llama-3.1-8B-Instruct"

In [10]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_Llama-3.1-8B_selected_interviews.csv", encoding="utf-8", sep=";")